In [2]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../Data/side_concordance_results.xlsx')


In [3]:
# def extract_side(location):
#     if pd.isnull(location):
#         return 'No Scan'
    
#     sides = []
    
#     if 'Left' in location:
#         sides.append('Left')
#     if 'Right' in location:
#         sides.append('Right')
    
#     if not sides:
#         return 'Unknown'
#     elif len(sides) == 1:
#         return sides[0]
#     else:
#         return 'Both'

def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'

In [4]:
# Define modalities and concordance column
spect_modality = 'SPECT/CT'
modality_4dct = '4D CT Scan'

# Apply extract_side to imaging and surgical findings
preop_imaging_df['4D CT Imaging Side'] = preop_imaging_df[modality_4dct].apply(extract_side)
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

negative_spect = preop_imaging_df[preop_imaging_df[spect_modality] == 'Non-localizing'].copy()

# Filter for cases with positive 4D CT localization
spect_neg_4d_pos = negative_spect[negative_spect['4D CT Imaging Side'].isin(['Left', 'Right'])].copy()


In [5]:
def check_concordance(row):
    imaging = row['4D CT Scan']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'


In [10]:
spect_neg_4d_pos['Calculated Concordance'] = spect_neg_4d_pos.apply(check_concordance, axis=1)

# Total cases
total_cases = spect_neg_4d_pos.shape[0]

# Count match types
exact_count = spect_neg_4d_pos[spect_neg_4d_pos['Calculated Concordance'] == 'Yes (Exact)'].shape[0]
side_count = spect_neg_4d_pos[spect_neg_4d_pos['Calculated Concordance'] == 'Yes (Side)'].shape[0]
incorrect_count = spect_neg_4d_pos[spect_neg_4d_pos['Calculated Concordance'] == 'Incorrect Localization'].shape[0]


exact_pct = (exact_count / total_cases) * 100 if total_cases > 0 else None
side_pct = (side_count / total_cases) * 100 if total_cases > 0 else None
incorrect_pct = (incorrect_count / total_cases) * 100 if total_cases > 0 else None



# Calculate combined side match count and percentage
total_side_match_count = exact_count + side_count
total_side_match_pct = (total_side_match_count / total_cases) * 100 if total_cases > 0 else None

# Create final summary dataframe with correct categories
spect_neg_4d_pos_summary_df = pd.DataFrame({
    'Match Type': [
        'Exact Match',
        'Total Side Match (Exact + Side-only)',
        'Incorrect Localization'
    ],
    'Count': [
        exact_count,
        total_side_match_count,
        incorrect_count
    ],
    'Percentage': [
        f"{exact_pct:.2f}%" if exact_pct is not None else 'N/A',
        f"{total_side_match_pct:.2f}%" if total_side_match_pct is not None else 'N/A',
        f"{incorrect_pct:.2f}%" if incorrect_pct is not None else 'N/A'
    ]
})

# Display results
print(f"Total SPECT negative / 4D CT positive cases: {total_cases}")
display(spect_neg_4d_pos_summary_df)

Total SPECT negative / 4D CT positive cases: 31


,Match Type,Count,Percentage
0,Exact Match,16,51.61%
1,Total Side Match (Exact + Side-only),20,64.52%
2,Incorrect Localization,11,35.48%


Procedure_Date            52
Patient_ID                52
Age                       51
Gender                    52
Surgical_Findings         52
Surgical Cure             52
SPECT/CT                  52
Ultrasound                10
4D CT Scan                48
Sestamibi                  2
MRI                        0
Ultrasound_Concordance    52
SPECT/CT_Concordance      52
4D CT Scan_Concordance    52
Sestamibi_Concordance     52
MRI_Concordance           52
4D CT Imaging Side        52
Surgical Side             52
dtype: int64